In [8]:
import requests as req
import pandas as pd

payload = {'download':'','hdn_gostartdate':'2017/09/1','hdn_goenddate':'2017/10/31','syear':'2017','smonth':'09','sday':'1','eyear':'2017','emonth':'10','eday':'31','datestart':'2017/09/1','dateend':'2017/10/31'}
html = req.post('http://www.taifex.com.tw/chinese/3/3_5.asp', data=payload)
#注意網頁編碼
html.encoding = 'utf-8'
df = pd.read_html(html.text)
df = df[2]
df.columns = df.iloc[0]
df = df.reindex(df.index.drop(0))
df.head()

,日期,美元／新台幣,人民幣／新台幣,歐元／美元,美元／日幣,英鎊／美元,澳幣／美元,美元／港幣,美元／人民幣,美元／南非幣,紐幣／美元
1,2017/9/1,30.156,4.584315,1.1892,110.1,1.29145,0.7936,7.82595,6.5781,12.9461,0.71595
2,2017/9/4,30.101,4.608885,1.1896,109.46,1.2942,0.7956,7.82535,6.5311,12.966,0.71705
3,2017/9/5,30.07,4.586457,1.1881,109.455,1.29255,0.79635,7.82525,6.55625,12.9729,0.71775
4,2017/9/6,30.105,4.606336,1.1944,108.71,1.3042,0.7991,7.8246,6.53555,12.90365,0.72335
5,2017/9/7,30.062,4.619808,1.19485,109.015,1.3046,0.80015,7.8196,6.5072,12.8172,0.719


In [3]:
df.to_csv('fp_demo2.csv', sep=',', encoding='utf-8-sig', index=False)

In [4]:
import pandas.io.sql as pd_sql
import sqlite3 as sql

#連線至sqlite檔案，若無該檔案sql，則會建立一個新的
conn = sql.connect("twse.db")
#將Dataframe資料寫入sql檔中的'demo2'表中，無該資料表則會自動建立
df.to_sql("demo2", conn, if_exists="replace")

In [9]:
pd.read_sql_query("select * from demo2;", conn).head()

,index,日期,美元／新台幣,人民幣／新台幣,歐元／美元,美元／日幣,英鎊／美元,澳幣／美元,美元／港幣,美元／人民幣,美元／南非幣,紐幣／美元
0,1,2017/9/1,30.156,4.584315,1.1892,110.1,1.29145,0.7936,7.82595,6.5781,12.9461,0.71595
1,2,2017/9/4,30.101,4.608885,1.1896,109.46,1.2942,0.7956,7.82535,6.5311,12.966,0.71705
2,3,2017/9/5,30.07,4.586457,1.1881,109.455,1.29255,0.79635,7.82525,6.55625,12.9729,0.71775
3,4,2017/9/6,30.105,4.606336,1.1944,108.71,1.3042,0.7991,7.8246,6.53555,12.90365,0.72335
4,5,2017/9/7,30.062,4.619808,1.19485,109.015,1.3046,0.80015,7.8196,6.5072,12.8172,0.719
